## 🧠 03_training_validation.ipynb – Entrenamiento y Validación de Modelos

Este notebook entrena y evalúa distintos modelos de clasificación para predecir la presencia de diabetes en pacientes, utilizando el dataset ya preprocesado.  
Se exploran modelos base, se comparan métricas clave y se registra el experimento con MLflow.

### 1. Carga del dataset limpio


Cargamos el archivo `diabetes_clean.csv` generado durante el preprocesamiento.  
Usamos `os.path.join` para construir rutas relativas seguras, ya que este notebook se encuentra en la carpeta `notebooks/` mientras que los datos están en `data/`.

> ⚠️ Si obtienes un error de archivo no encontrado (`FileNotFoundError`), asegúrate de:
> - Haber ejecutado completamente el notebook `02_preprocessing.ipynb`
> - Estar en el directorio raíz del proyecto al iniciar el entorno de notebooks

In [4]:
import pandas as pd
import os

# Construcción de ruta robusta al archivo limpio
file_path = os.path.join("..", "data", "diabetes_clean.csv")
df = pd.read_csv(file_path)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome,Insulin
0,6,148.0,72.0,35.0,33.6,0.627,50,1,1.764094
1,1,85.0,66.0,29.0,26.6,0.351,31,0,1.764094
2,8,183.0,64.0,29.0,23.3,0.672,32,1,1.764094
3,1,89.0,66.0,23.0,28.1,0.167,21,0,1.714496
4,0,137.0,44.0,35.0,43.1,2.288,33,1,1.813178


### 🔄 2. Separación de variables independientes (X) y variable objetivo (y)

En este paso separamos la variable objetivo `Outcome` del resto de los predictores.


In [8]:
# Separar variables predictoras (X) y variable objetivo (y)
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

### 🧪 3. División del dataset en entrenamiento y prueba

Dividimos el dataset en un conjunto de entrenamiento (80%) y uno de prueba (20%), manteniendo la proporción de clases mediante `stratify=y`.  
Esta división se hace **antes** de ajustar cualquier modelo para evitar data leakage.

In [10]:
from sklearn.model_selection import train_test_split

# División 80/20 con estratificación
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

### 🧾 4. Verificación de dimensiones

Revisamos las dimensiones de los conjuntos para confirmar que la división fue exitosa y las proporciones se mantuvieron.

In [11]:
print("Shape de X_train:", X_train.shape)
print("Shape de X_test:", X_test.shape)
print("Shape de y_train:", y_train.shape)
print("Shape de y_test:", y_test.shape)

Shape de X_train: (614, 8)
Shape de X_test: (154, 8)
Shape de y_train: (614,)
Shape de y_test: (154,)


### 5. 🔄 Escalado de características

Aplicamos `StandardScaler` para escalar las variables predictoras. Esta técnica transforma los datos para que tengan media 0 y desviación estándar 1. Es importante ajustar el escalador únicamente con el conjunto de entrenamiento para evitar data leakage.

Transformamos `X_train` y `X_test` usando este escalador.

In [12]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar solo con datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transformar también el conjunto de prueba
X_test_scaled = scaler.transform(X_test)

print("Media (X_train_scaled):", np.mean(X_train_scaled, axis=0))
print("Desviación estándar (X_train_scaled):", np.std(X_train_scaled, axis=0))

Media (X_train_scaled): [-6.94341436e-17 -1.09937394e-16 -1.66352636e-16 -6.94341436e-17
  6.27800381e-16 -1.09937394e-16 -1.08490849e-16 -5.11353536e-16]
Desviación estándar (X_train_scaled): [1. 1. 1. 1. 1. 1. 1. 1.]



#### 💾 Guardado de datos procesados

Se almacenan los conjuntos de datos `X_train_scaled`, `X_test_scaled`, `y_train`, `y_test` como archivos `.pkl` para reutilizarlos en el notebook `04_evaluation_export.ipynb` sin tener que repetir el preprocesamiento.

In [14]:
import os
import joblib

# Crear carpeta ../data si no existe
os.makedirs('../data', exist_ok=True)

# Guardar datasets escalados desde notebooks/
joblib.dump(X_train_scaled, '../data/X_train_scaled.pkl')
joblib.dump(X_test_scaled, '../data/X_test_scaled.pkl')
joblib.dump(y_train, '../data/y_train.pkl')
joblib.dump(y_test, '../data/y_test.pkl')

print("✅ Datos escalados guardados exitosamente en ../data/")

✅ Datos escalados guardados exitosamente en ../data/




#### ✅ Conclusiones de la etapa de validación y escalado

- Se realizó una correcta división del dataset en entrenamiento y prueba con una proporción 80/20, utilizando estratificación para conservar la proporción de clases en ambos subconjuntos.
- Se aplicó `StandardScaler` exclusivamente con los datos de entrenamiento y luego se transformaron los datos de prueba, siguiendo buenas prácticas para evitar filtración de datos.
- Se verificó que los datos escalados tienen media cercana a 0 y desviación estándar igual a 1.
- Finalmente, se guardaron los conjuntos escalados en archivos `.pkl` utilizando `joblib` para su reutilización posterior en el módulo `04_evaluation_export.ipynb`.

Este enfoque modular facilita el entrenamiento y la evaluación de modelos de manera eficiente, reproducible y alineada con buenas prácticas de ciencia de datos.

